# Lesson 3: Reflection and Blogpost Writing

## Setup

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
llm_config = {"model":"gpt-4.1"}


## The task!

In [5]:
task = '''
        Write a concise but engaging blogpost about 
        AI and the Future of Jobs . Make sure the blogpost is
        within 100 words.
        '''


## Create a writer agent

In [6]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [7]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [8]:
print(reply)

AI and the Future of Jobs: Embracing Change

Artificial intelligence is revolutionizing the workplace, automating repetitive tasks and sparking concerns about job security. However, rather than eliminating work, AI is transforming it—creating demand for new skills like data analysis, digital collaboration, and creative problem-solving. Workers who embrace lifelong learning and adapt to technological shifts will thrive in this evolving landscape. By leveraging AI as a tool, we can unlock human potential and shape a future where technology and talent work hand in hand to drive innovation and growth. The key is not to fear change, but to prepare for it.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

![AI agents Reflection](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/10/image-1-1-1.webp)

In [9]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [10]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about 
        AI and the Future of Jobs . Make sure the blogpost is
        within 100 words.
        

--------------------------------------------------------------------------------
Writer (to Critic):

AI and the Future of Jobs: Embracing the Shift

Artificial intelligence is rapidly transforming workplaces worldwide. While some fear job loss, AI is also creating new opportunities in tech, healthcare, education, and beyond. Routine tasks are increasingly automated, freeing workers for creative, problem-solving roles. Adaptability is key: by learning new skills and embracing lifelong education, today’s workforce can thrive alongside AI innovations. Instead of replacing humans, AI promises a future where collaboration between people and machines drives unprecedented progress and prosperity. The real challenge is preparing for this shift—future-proofing our talents for a rapidly evolving world of work.

-------------

## Nested chat

In [11]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [12]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=3,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about 
        AI and the Future of Jobs . Make sure the blogpost is
        within 100 words.
        

--------------------------------------------------------------------------------
Writer (to Critic):

AI and the Future of Jobs: Adapting to a New Era

Artificial intelligence is revolutionizing the workplace, automating routine tasks and prompting fears of job loss. But AI also creates new opportunities, demanding skills in data analysis, creativity, and critical thinking. The workforce must pivot, embracing lifelong learning and adaptability. Instead of replacing humans, AI can augment our abilities, making work more efficient and meaningful. The future belongs to those ready to evolve with technology—where man and machine, together, fuel innovation. By adapting now, we can thrive in an AI-powered world and unlock unprecedented potential for both individuals and industries.

---------------------------------------

## Get the summary

In [18]:
print(res.summary)

Meta Description:  
Explore how AI is influencing the future of work, its impact on jobs, and inclusive strategies for AI skills development. Learn practical steps to thrive in the AI era.

---

## AI and Jobs: Shaping the Future of Work with Artificial Intelligence

Artificial intelligence is rapidly transforming industries worldwide. Recent studies estimate that by 2030, up to 375 million workers may need to switch occupational categories due to AI and automation (McKinsey, 2023). Instead of replacing all jobs, AI is shifting the skills landscape, placing a premium on adaptability, data literacy, and creative problem-solving.

### The Impact of AI on Jobs

AI’s impact on jobs is complex. While it automates repetitive and menial tasks, it also creates new roles—especially in data analysis, machine learning, and AI governance. According to the World Economic Forum, 97 million new jobs could emerge by 2025 as businesses reimagine roles alongside AI technologies.

### Key Strategies for 

In [ ]:

print(res.cost)

{'usage_including_cached_inference': {'total_cost': 0.017240000000000002, 'gpt-4.1-2025-04-14': {'cost': 0.017240000000000002, 'prompt_tokens': 2800, 'completion_tokens': 1455, 'total_tokens': 4255}}, 'usage_excluding_cached_inference': {'total_cost': 0.017240000000000002, 'gpt-4.1-2025-04-14': {'cost': 0.017240000000000002, 'prompt_tokens': 2800, 'completion_tokens': 1455, 'total_tokens': 4255}}}
